# Testing snow product with Fmask - single tile testing
- After finishing IMS work and related updates

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime as dt
import hvplot.xarray
import os

from dask.distributed import LocalCluster
from skimage.measure import label

import sys
sys.path.append('C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/UtilityCode/DataAccess/Utilities')
import PreProcess_Utils as pputil

sys.path.append('C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/UtilityCode/Modelling/Utilities')
import Snow_Utils as snow_util

#import holoviews as hv

In [2]:
# hv.extension("matplotlib")
# hv.extension("bokeh")

In [3]:
cluster = LocalCluster(n_workers = 10, threads_per_worker = 10) # Refined through testing
client = cluster.get_client()
client # Can open in other window by clicking 'Dashboard' link below

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 100,Total memory: 127.65 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:63751,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 100
Started: Just now,Total memory: 127.65 GiB
Comm: tcp://127.0.0.1:63818,Total threads: 10
Dashboard: http://127.0.0.1:63830/status,Memory: 12.77 GiB
Nanny: tcp://127.0.0.1:63754,


In [4]:
tile_stats = pd.read_csv(r"C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\High Resolution Snow Dynamics\IMS (1 km)\Outputs\tiles_IMS_SnowDynamics4HLS.csv", index_col = 0)
tile_stats # Note, not every tile (snow free removed)

,smin_avg,smax_avg,sstart_avg,send_avg,send_last,sstart_first
0,212.16667,36.6,3.000000,67.000000,90.0,335.0
1,219.50000,39.6,1.800000,73.600000,90.0,335.0
2,211.66667,36.6,9.800000,66.400000,90.0,335.0
3,196.33333,35.6,35.333332,69.333336,90.0,335.0
4,211.83333,39.4,5.200000,65.200000,90.0,335.0
...,...,...,...,...,...,...
3386,232.00000,39.2,-132.000000,232.000000,NaN,NaN
3387,227.83333,40.2,-136.166670,227.833330,NaN,NaN
3388,225.00000,37.6,-139.000000,225.000000,NaN,NaN
3389,230.33333,33.6,-133.666670,230.333330,NaN,NaN


# Load Fmask cubes

In [5]:
tile = 904 # From canadaFishnet_CanVec_60km
w_s = 2022 # Year that winter ends
w_e = 2023 # Year that winter starts
winterYear = str(w_s)[2:] + str(w_e)[2:] 

path = 'F:/HLS/Canada/Tiles60km/'

In [6]:
%%time
c_s = pputil.uploadNC(path + str(w_s) + '/fmask_' + str(tile) + '_' + str(w_s) + '.nc', 
                      type = 'dataarray',
                      chunks = {'time': 1, 'band': -1, 'x': -1, 'y': -1})
c_s

CPU times: total: 156 ms
Wall time: 1.52 s


<xarray.DataArray 'stackstac-1b087a35d8fb3852cd9d76a26e8df9e9' (time: 209,
                                                                band: 1,
                                                                y: 2001, x: 2001)> Size: 837MB
dask.array<open_dataset-stackstac-1b087a35d8fb3852cd9d76a26e8df9e9, shape=(209, 1, 2001, 2001), dtype=uint8, chunksize=(1, 1, 2001, 2001), chunktype=numpy.ndarray>
Coordinates:
  * band           (band) <U5 20B 'Fmask'
  * x              (x) float64 16kB 6.49e+05 6.49e+05 ... 7.089e+05 7.09e+05
  * y              (y) float64 16kB 6.553e+05 6.552e+05 ... 5.953e+05 5.953e+05
    constellation  (time) <U3 3kB dask.array<chunksize=(1,), meta=np.ndarray>
  * time           (time) datetime64[ns] 2kB 2022-01-13 ... 2022-12-01
Attributes:
    spec:        RasterSpec(epsg=3979, bounds=(648960, 595260, 708990, 655290...
    crs:         epsg:3979
    transform:   [ 3.0000e+01  0.0000e+00  6.4896e+05  0.0000e+00 -3.0000e+01...
    resolution:  30

In [7]:
%%time
c_e = pputil.uploadNC(path + str(w_e) + '/fmask_' + str(tile) + '_' + str(w_e) + '.nc', 
                      type = 'dataarray',
                      chunks = {'time': 1, 'band': -1, 'x': -1, 'y': -1})
c_e

CPU times: total: 0 ns
Wall time: 139 ms


<xarray.DataArray 'stackstac-cedc179474b6d33c85a1442abfd5fa7e' (time: 199,
                                                                band: 1,
                                                                y: 2001, x: 2001)> Size: 797MB
dask.array<open_dataset-stackstac-cedc179474b6d33c85a1442abfd5fa7e, shape=(199, 1, 2001, 2001), dtype=uint8, chunksize=(1, 1, 2001, 2001), chunktype=numpy.ndarray>
Coordinates:
  * band           (band) <U5 20B 'Fmask'
  * x              (x) float64 16kB 6.49e+05 6.49e+05 ... 7.089e+05 7.09e+05
  * y              (y) float64 16kB 6.553e+05 6.552e+05 ... 5.953e+05 5.953e+05
    constellation  (time) <U3 2kB dask.array<chunksize=(1,), meta=np.ndarray>
  * time           (time) datetime64[ns] 2kB 2023-01-13 ... 2023-12-01
Attributes:
    spec:        RasterSpec(epsg=3979, bounds=(648960, 595260, 708990, 655290...
    crs:         epsg:3979
    transform:   [ 3.0000e+01  0.0000e+00  6.4896e+05  0.0000e+00 -3.0000e+01...
    resolution:  30

# Convert to snow cube

In [8]:
%%time
snowCube = snow_util.annualFmask2SnowCube(c_s, c_e, yrs = 2, verbose = True) # yrs = 1, doy = doy
# Want to use 2 year cube to help get better results due to date gaps
snowCube

Initial Fmask time-steps: Start (n = 209), End (n = 199).
Made decision on same-day observations (L30vS30): Start (n = 167), End (n = 164).
Combined snow cube (n = 331).
Removed time-steps above 99% unclear pixels (n = 220).
Reclassifed (Snow = 1, Non-snow = 0, Unclear = NaN)
CPU times: total: 2.58 s
Wall time: 21.9 s


<xarray.DataArray 'stackstac-1b087a35d8fb3852cd9d76a26e8df9e9' (time: 220,
                                                                y: 2001, x: 2001)> Size: 4GB
dask.array<where, shape=(220, 2001, 2001), dtype=float32, chunksize=(1, 2001, 2001), chunktype=numpy.ndarray>
Coordinates:
    band         <U5 20B 'Fmask'
  * x            (x) float64 16kB 6.49e+05 6.49e+05 ... 7.089e+05 7.09e+05
  * y            (y) float64 16kB 6.553e+05 6.552e+05 ... 5.953e+05 5.953e+05
  * time         (time) datetime64[ns] 2kB 2022-01-20 2022-01-23 ... 2023-12-01
    spatial_ref  int64 8B 0

# Get Observation Frequency (clear observations per day)
- Using IMS: winter year (smin to smin), snow season (1 week before snow_start to 1 week after snow_end), snow fall period (1 week before to 6 weeks after snow_start), winter peak (7 weeks around smax) snow melt period (6 weeks before to 1 week after snow_end)
- Can do along with tile level results from Fmaks observation availability project

In [9]:
%%time
# clearObsPerDay = snow_util.clearObservationsPerDay_WinterYear(snowCube, smin_doy = smin_doy, smax_doy = smax_doy, 
#                                                               sstart_doy = sstart_doy, send_doy = send_doy)
clearObsPerDay = snow_util.clearObservationsPerDay_WinterYear(snowCube,
                                                              windows = ['snowSeason', 'snowFall', 'snowMelt'],
                                                              smin_doy = int(tile_stats.loc[tile, 'smin_avg']),
                                                              #smax_doy = int(tile_stats.loc[tile, 'smax_avg']), # Only needed for snowPeak
                                                              sstart_doy = int(tile_stats.loc[tile, 'sstart_avg']),
                                                              send_doy = int(tile_stats.loc[tile, 'send_avg']))
clearObsPerDay

Created empty clearObsPerDay Dataset to fill.
Added snow season (2022-10-19 to 2023-06-08) to clearObsPerDay.
Added snow fall period (2022-10-19 to 2022-12-06) to clearObsPerDay.
Added snow melt period (2023-04-21 to 2023-06-08) to clearObsPerDay.
CPU times: total: 125 ms
Wall time: 336 ms


<xarray.Dataset> Size: 48MB
Dimensions:                    (x: 2001, y: 2001)
Coordinates:
  * x                          (x) float64 16kB 6.49e+05 6.49e+05 ... 7.09e+05
  * y                          (y) float64 16kB 6.553e+05 ... 5.953e+05
    spatial_ref                int64 8B 0
    band                       <U5 20B 'Fmask'
Data variables:
    snowSeason_clearObsPerDay  (y, x) float32 16MB dask.array<chunksize=(2001, 2001), meta=np.ndarray>
    snowFall_clearObsPerDay    (y, x) float32 16MB dask.array<chunksize=(2001, 2001), meta=np.ndarray>
    snowMelt_clearObsPerDay    (y, x) float32 16MB dask.array<chunksize=(2001, 2001), meta=np.ndarray>

In [10]:
%%time
clearObsPerDay = pputil.loadXR(clearObsPerDay)
clearObsPerDay

CPU times: total: 797 ms
Wall time: 5.35 s


<xarray.Dataset> Size: 48MB
Dimensions:                    (x: 2001, y: 2001)
Coordinates:
  * x                          (x) float64 16kB 6.49e+05 6.49e+05 ... 7.09e+05
  * y                          (y) float64 16kB 6.553e+05 ... 5.953e+05
    spatial_ref                int64 8B 0
    band                       <U5 20B 'Fmask'
Data variables:
    snowSeason_clearObsPerDay  (y, x) float32 16MB 0.1202 0.1202 ... 0.1588
    snowFall_clearObsPerDay    (y, x) float32 16MB 0.04082 0.04082 ... 0.08163
    snowMelt_clearObsPerDay    (y, x) float32 16MB 0.1633 0.1633 ... 0.2245

In [11]:
#clearObsPerDay['snowFall'].plot()

In [12]:
#clearObsPerDay['snowMelt'].plot()

In [13]:
# clearObsPerDay.nbytes / 1e9

In [14]:
# %%time
# pputil.downloadNC(clearObsPerDay, 'C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/HBL/TemporalSnow/Fmask_V1/Testing', \
#                   'clearObsPerDay_winterYear' + str(w_s)[2:] + str(w_e)[2:] + '.nc', type = '')

# Get Snow Dynamics

In [15]:
thresh = 2 # Originally 3 but in some situations leads to extra uncertainty


if np.isnan(tile_stats.loc[tile, 'send_last']) == True:
    implausible_snow = []
else: # Week buffer from latest recorded end date and earliest recorded start date
    implausible_snow = [int(tile_stats.loc[tile, 'send_last']) + 7, int(tile_stats.loc[tile, 'sstart_first']) - 7]

implausible_snow

[163, 272]

In [16]:
%%time
snowCube_c = snow_util.cleanSnowCube(snowCube, implausible_snow = implausible_snow, thresh = thresh)
snowCube_c

Set all observations to non-snow between implausible snow dates (06-12 to 09-29).
Calculated cumulative sum that resets when non-snow is found.
Identified periods when 2+ consecutive snow observations occured (skipping NaN).
Created flipped snowCube to identify non-snow periods.
Calculated cumulative sum that resets when snow is found.
Identified periods when 2+ consecutive non-snow observations occured (skipping NaN).
Removed (NaN) snow and non-snow periods below 2 consecutive observations.
CPU times: total: 62.5 ms
Wall time: 298 ms


<xarray.DataArray (time: 220, y: 2001, x: 2001)> Size: 4GB
dask.array<where, shape=(220, 2001, 2001), dtype=float32, chunksize=(1, 2001, 2001), chunktype=numpy.ndarray>
Coordinates:
    band         <U5 20B 'Fmask'
  * x            (x) float64 16kB 6.49e+05 6.49e+05 ... 7.089e+05 7.09e+05
  * y            (y) float64 16kB 6.553e+05 6.552e+05 ... 5.953e+05 5.953e+05
  * time         (time) datetime64[ns] 2kB 2022-01-20 2022-01-23 ... 2023-12-01
    spatial_ref  int64 8B 0

In [17]:
%%time
snowCube_c = snow_util.dailySnowCube(snowCube_c, doy = int(tile_stats.loc[tile, 'smin_avg']), verbose = True)
snowCube_c

Filled in all NaNs in snowCube with linearly interpolated values in time.
Resampled to daily snowCube (2022-01-20 to 2023-12-01, n = 681) using linear interpolation.
Filtered to winter year (2022-08-14 to 2023-08-13, n = 365).
Reinidexing not required since daily snowCube already contains all possible dates.
CPU times: total: 15.6 ms
Wall time: 85.8 ms


<xarray.DataArray (time: 365, y: 2001, x: 2001)> Size: 6GB
dask.array<getitem, shape=(365, 2001, 2001), dtype=float32, chunksize=(365, 390, 390), chunktype=numpy.ndarray>
Coordinates:
    band         <U5 20B 'Fmask'
  * x            (x) float64 16kB 6.49e+05 6.49e+05 ... 7.089e+05 7.09e+05
  * y            (y) float64 16kB 6.553e+05 6.552e+05 ... 5.953e+05 5.953e+05
    spatial_ref  int64 8B 0
  * time         (time) datetime64[ns] 3kB 2022-08-14 2022-08-15 ... 2023-08-13

In [18]:
%%time
snowDynamics = snow_util.dailySnowCube2SnowDynamics(snowCube_c, uncertainty = True, verbose = True) # Default settings
#snowDynamics = snow_util.dailySnowCube2SnowDynamics(snowCube_c, products = ['start', 'end', 'endSum'], uncertainty = True, verbose = True) # snow_endSum
snowDynamics

Created empty snowDynamics Dataset to fill.
Created binary snowCube (>= 0.5 = 1, < 0.5 = 0).
Calculated cumulative sum of binary snowCube (used in multiple products).
Added snow cover start date (# days from Dec 31) to snowDynamics.
Added snow cover end date (# days from Dec 31) to snowDynamics.
Added snow cover length (# days with snow cover) to snowDynamics.
Added snow period count (# seperated snow periods) to snowDynamics.
Added snow status (0 = Regular fall/melt, 1 = Perennial, 2 = Inconsistent perennial, 3 = Snow free) to snowDynamics.
Created uncertainty snowCube (>= 0.005 & <= 0.995 = 1, else = 0).
Used 77 day gap between latest snow start (2023-02-08) and earliest snow end (2023-04-26) to find winter year split day: 2023-03-18.
Added snow start uncertainty (# days between last non-snow and first snow observation / 2) to snowDynamics.
Added snow end uncertainty (# days between last snow and first non-snow observation / 2) to snowDynamics.
Added snow length uncertainty (total # 

<xarray.Dataset> Size: 96MB
Dimensions:        (x: 2001, y: 2001)
Coordinates:
  * x              (x) float64 16kB 6.49e+05 6.49e+05 ... 7.089e+05 7.09e+05
  * y              (y) float64 16kB 6.553e+05 6.552e+05 ... 5.953e+05 5.953e+05
    spatial_ref    int64 8B 0
    band           <U5 20B 'Fmask'
Data variables:
    snow_start     (y, x) float32 16MB dask.array<chunksize=(390, 390), meta=np.ndarray>
    snow_end       (y, x) float32 16MB dask.array<chunksize=(390, 390), meta=np.ndarray>
    snow_length    (y, x) uint16 8MB dask.array<chunksize=(390, 390), meta=np.ndarray>
    snow_periods   (y, x) uint8 4MB dask.array<chunksize=(390, 390), meta=np.ndarray>
    snow_status    (y, x) uint8 4MB dask.array<chunksize=(390, 390), meta=np.ndarray>
    snow_start_u   (y, x) float32 16MB dask.array<chunksize=(390, 390), meta=np.ndarray>
    snow_end_u     (y, x) float32 16MB dask.array<chunksize=(390, 390), meta=np.ndarray>
    snow_length_u  (y, x) float32 16MB dask.array<chunksize=(390, 390), meta=np.ndarray>

# Merge and Save

In [19]:
%%time
## Add observations per day to snowDynamics
output = xr.merge([snowDynamics, clearObsPerDay])
output

CPU times: total: 0 ns
Wall time: 1.52 ms


<xarray.Dataset> Size: 144MB
Dimensions:                    (x: 2001, y: 2001)
Coordinates:
  * x                          (x) float64 16kB 6.49e+05 6.49e+05 ... 7.09e+05
  * y                          (y) float64 16kB 6.553e+05 ... 5.953e+05
    spatial_ref                int64 8B 0
    band                       <U5 20B 'Fmask'
Data variables:
    snow_start                 (y, x) float32 16MB dask.array<chunksize=(390, 390), meta=np.ndarray>
    snow_end                   (y, x) float32 16MB dask.array<chunksize=(390, 390), meta=np.ndarray>
    snow_length                (y, x) uint16 8MB dask.array<chunksize=(390, 390), meta=np.ndarray>
    snow_periods               (y, x) uint8 4MB dask.array<chunksize=(390, 390), meta=np.ndarray>
    snow_status                (y, x) uint8 4MB dask.array<chunksize=(390, 390), meta=np.ndarray>
    snow_start_u               (y, x) float32 16MB dask.array<chunksize=(390, 390), meta=np.ndarray>
    snow_end_u                 (y, x) float32 16MB dask.array<chunksize=(390, 390), meta=np.ndarray>
    snow_length_u              (y, x) float32 16MB dask.array<chunksize=(390, 390), meta=np.ndarray>
    snowSeason_clearObsPerDay  (y, x) float32 16MB 0.1202 0.1202 ... 0.1588
    snowFall_clearObsPerDay    (y, x) float32 16MB 0.04082 0.04082 ... 0.08163
    snowMelt_clearObsPerDay    (y, x) float32 16MB 0.1633 0.1633 ... 0.2245

In [20]:
# Save examples to tif for documentation
# output['snow_start'].rio.to_raster('snowStart_2223_904.tif')
# output['snow_end'].rio.to_raster('snowEnd_2223_904.tif')
#output['snow_endSum'].rio.to_raster('snowEndSum_2223_904.tif')
# output['snow_length'].rio.to_raster('snowLength_2223_904.tif')
# output['snow_periods'].rio.to_raster('snowPeriods_2223_904.tif')
#output['snow_status'].rio.to_raster('snowStatus_2223_904.tif')
# output['snow_start_u'].rio.to_raster('snowStartU_2223_904.tif')
# output['snow_end_u'].rio.to_raster('snowEndU_2223_904.tif')
# output['snow_length_u'].rio.to_raster('snowLengthU_2223_904.tif')

In [21]:
#output.nbytes / 1e9 # 0.15 GB

In [22]:
# %%time
# output = output.expand_dims(winterYear = xr.Variable('winterYear', [str(w_s) + '-' + str(w_e)])) # Winter year as time dimension

# path = r'C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\HBL\TemporalSnow\Fmask_V1\Products\winterYear_tile' + '/' + str(tile)
# if not os.path.exists(path): # Create folder to save in if needed
#     os.makedirs(path)

# pputil.downloadNC(output, path, 'HLS_Fmask_snowDynamics_winterYear' + winterYear + '.nc', type = '')

## Does snow dynamics need to be cleaned more by IMS? 
- Going to use weighted uncertainty, so no more need to clean by that...

In [23]:
# tile_stats.loc[tile]

In [24]:
# implausible_snow

In [25]:
# implausible_snow1 = [implausible_snow[0] - len(snowCube_c), implausible_snow[1] - len(snowCube_c)]
# implausible_snow1

In [26]:
# implausible_length = len(snowCube_c) - (implausible_snow[1] - implausible_snow[0])
# implausible_length

### snow_start

In [27]:
# snowDynamics['snow_start'].plot()

In [28]:
# snowDynamics['snow_start'].plot.hist()
# print(int(snowDynamics['snow_start'].min()))
# print(int(snowDynamics['snow_start'].mean()))
# print(int(snowDynamics['snow_start'].max()))
# print(int(snowDynamics['snow_start'].isnull().sum()))

### snow_start_u

In [29]:
# snowDynamics['snow_start_u'].plot()

In [30]:
# snowDynamics['snow_start_u'].plot.hist()
# print(int(snowDynamics['snow_start_u'].min()))
# print(int(snowDynamics['snow_start_u'].mean()))
# print(int(snowDynamics['snow_start_u'].max()))
# print(int(snowDynamics['snow_start_u'].isnull().sum()))

### snow_end

In [31]:
#snowDynamics['snow_end'].plot()

In [32]:
# snowDynamics['snow_end'].plot.hist()
# print(int(snowDynamics['snow_end'].min()))
# print(int(snowDynamics['snow_end'].mean()))
# print(int(snowDynamics['snow_end'].max()))
# print(int(snowDynamics['snow_end'].isnull().sum()))

In [33]:
# # Del stuff to save memory # Does not help...
# del c_s, c_e, snowCube, clearObsPerDay, snowCube_c, snowDynamics, output
# gc.collect()

### snow_end_u

In [34]:
# snowDynamics['snow_end_u'].plot()

In [35]:
# snowDynamics['snow_end_u'].plot.hist()
# print(int(snowDynamics['snow_end_u'].min()))
# print(int(snowDynamics['snow_end_u'].mean()))
# print(int(snowDynamics['snow_end_u'].max()))
# print(int(snowDynamics['snow_end_u'].isnull().sum()))

### snow_length

In [36]:
# snowDynamics['snow_length'].plot()

In [37]:
# snowDynamics['snow_length'].plot.hist()
# print(int(snowDynamics['snow_length'].min()))
# print(int(snowDynamics['snow_length'].mean()))
# print(int(snowDynamics['snow_length'].max()))
# print(int(snowDynamics['snow_length'].isnull().sum()))

### snow_length_u

In [38]:
# snowDynamics['snow_length_u'].plot()

In [39]:
# snowDynamics['snow_length_u'].plot.hist()
# print(int(snowDynamics['snow_length_u'].min()))
# print(int(snowDynamics['snow_length_u'].mean()))
# print(int(snowDynamics['snow_length_u'].max()))
# print(int(snowDynamics['snow_length_u'].isnull().sum()))

### snow_periods

In [40]:
# snowDynamics['snow_periods'].plot()

In [41]:
# np.unique(snowDynamics['snow_periods'], return_counts = True)

### snow_status

In [42]:
# snowDynamics['snow_status'].plot()

In [43]:
# np.unique(snowDynamics['snow_status'], return_counts = True)

# Snow Period ID/Cleaning Validation
- Interpret 100 random points and get snow dynamics
- 20 from each winter year
- See 100pixel_SnowDynamics.xlsx

Find method that best captures our interpretation

In [44]:
%%time
# Loading here will save time when getting plots for 100 points
snowCube = pputil.loadXR(snowCube)
snowCube

CPU times: total: 13 s
Wall time: 26.8 s


<xarray.DataArray 'stackstac-1b087a35d8fb3852cd9d76a26e8df9e9' (time: 220,
                                                                y: 2001, x: 2001)> Size: 4GB
array([[[ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        ...,
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.]],

       [[ 1.,  1., nan, ...,  1.,  1.,  1.],
        [ 1.,  1., nan, ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        ...,
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ...,  1.,  1.,  1.],
        [ 1.,  1., nan, ...,  1.,  1.,  1.],
        [ 1.,  1., nan, ...,  1.,  1.,  1.]],

       [[nan, nan, nan, ...,  1.,  1.,  1.],
        [nan, nan, nan, ...,  1.,  1.,  1.],
        [nan, nan, nan, ...,  1.,  1.,  1.],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ...,  1.,  1.,  1.],
        [nan, nan, nan, ...,  1.,  1.,  1.],
        [nan, nan, nan, ...,  1.,  1.,  1.]]], dtype=float32)
Coordinates:
    band         <U5 20B 'Fmask'
  * x            (x) float64 16kB 6.49e+05 6.49e+05 ... 7.089e+05 7.09e+05
  * y            (y) float64 16kB 6.553e+05 6.552e+05 ... 5.953e+05 5.953e+05
  * time         (time) datetime64[ns] 2kB 2022-01-20 2022-01-23 ... 2023-12-01
    spatial_ref  int64 8B 0

In [45]:
x = 1208 #np.random.randint(0, snowCube.shape[1])
print(x)
y = 1845 #np.random.randint(0, snowCube.shape[2])
print(y)

print(snowCube[:, x, y]['x'].values) #.rio.set_spatial_dims('x', 'y').rio.reproject('EPSG:4326')
print(snowCube[:, x, y]['y'].values)

(snowCube[:, x, y].rename('Snow Cube') # To plot with hvplot
                  .dropna(dim = 'time') # Prepare for daily resample
                  .resample(time = '1D').interpolate('linear').astype('float32') # Linear daily resample
                  # Reindex to fill 2 year time-series?
                  .hvplot(height = 600, width = 1500) * # Drop na and interpolate na to see full picture

(snowCube[:, x, y].hvplot.scatter(marker = 'o', color = 'black', size = 15))) # Observation points

1208
1845
704325.0
619035.0


:Overlay
   .Curve.I   :Curve   [time]   (Snow Cube)
   .Scatter.I :Scatter   [time]   (stackstac-1b087a35d8fb3852cd9d76a26e8df9e9)

In [46]:
# dates = snowCube[:, x, y].time.values
# dates = [date[0:10] for date in np.datetime_as_string(dates)]

# obs = snowCube[:, x, y].values
# obs = [ob.item() for ob in obs]

# {dates: obs for dates, obs in zip(dates, obs)}

In [47]:
#snowCube[:, x, y]

## Method

### cleanSnowCube
- Resetting cumsum, threshold = 3

In [48]:
if np.isnan(tile_stats.loc[tile, 'send_last']) == True:
    implausible_snow = []
else: # Week buffer from latest recorded end date and earliest recorded start date
    implausible_snow = [int(tile_stats.loc[tile, 'send_last']) + 7, int(tile_stats.loc[tile, 'sstart_first']) - 7]
implausible_snow

[163, 272]

In [49]:
snowCube[:, x, y]

<xarray.DataArray 'stackstac-1b087a35d8fb3852cd9d76a26e8df9e9' (time: 220)> Size: 880B
array([ 1.,  1., nan,  1., nan,  1., nan,  1.,  1.,  1.,  1., nan,  1.,
       nan,  1., nan,  1.,  1., nan,  1.,  1., nan,  1.,  1.,  1.,  1.,
       nan,  1.,  1.,  1.,  1., nan,  1., nan,  1.,  1.,  1.,  1.,  1.,
       nan,  1.,  1.,  1., nan, nan, nan,  0.,  0., nan, nan, nan,  0.,
        0.,  0., nan, nan,  0., nan,  0., nan,  0., nan, nan,  0.,  0.,
       nan, nan,  0.,  0., nan, nan,  0.,  0.,  0.,  0., nan, nan,  0.,
       nan, nan,  0.,  0.,  0.,  0., nan, nan,  0., nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan,  0.,  0., nan, nan, nan,  1.,
       nan, nan,  1., nan,  1., nan, nan,  1.,  1., nan, nan, nan,  1.,
        1., nan,  1.,  1., nan, nan,  1.,  1.,  1., nan,  1.,  1.,  1.,
        1.,  1., nan,  1., nan,  1., nan,  1.,  1.,  1.,  1.,  1.,  0.,
       nan,  1., nan,  1.,  1., nan, nan,  1.,  1.,  1.,  1., nan,  0.,
        0.,  0.,  0.,  0., nan,  0.,  0., nan, nan,  0., nan,  0.,  0.,
        0.,  0., nan,  0.,  0., nan, nan,  0., nan, nan,  0.,  0.,  0.,
       nan, nan, nan,  0.,  1., nan, nan,  0.,  0., nan,  0.,  0., nan,
       nan,  0.,  0., nan,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0., nan,
       nan, nan, nan,  0.,  0., nan, nan, nan,  0.,  1.,  1.,  1.],
      dtype=float32)
Coordinates:
    band         <U5 20B 'Fmask'
    x            float64 8B 7.043e+05
    y            float64 8B 6.19e+05
  * time         (time) datetime64[ns] 2kB 2022-01-20 2022-01-23 ... 2023-12-01
    spatial_ref  int64 8B 0

In [50]:
%%time

thresh = 2

snowCube_c = snow_util.cleanSnowCube(snowCube[:, x-5:x+5, y-5:y+5], implausible_snow = implausible_snow, thresh = thresh)
#snowCube_c

(snowCube_c[:, 5, 5].rename('Snow Cube') # To plot with hvplot
                    .dropna(dim = 'time') # Prepare for daily resample
                    .resample(time = '1D').interpolate('linear').astype('float32') # Linear daily resample
                    # Reindex to fill 2 year time-series?
                    .hvplot(height = 600, width = 1500) * # Drop na and interpolate na to see full picture
 
(snowCube_c[:, 5, 5].rename('s').hvplot.scatter(marker = 'o', color = 'black', size = 15))) # Observation points

Set all observations to non-snow between implausible snow dates (06-12 to 09-29).
Calculated cumulative sum that resets when non-snow is found.
Identified periods when 2+ consecutive snow observations occured (skipping NaN).
Created flipped snowCube to identify non-snow periods.
Calculated cumulative sum that resets when snow is found.
Identified periods when 2+ consecutive non-snow observations occured (skipping NaN).
Removed (NaN) snow and non-snow periods below 2 consecutive observations.
CPU times: total: 62.5 ms
Wall time: 197 ms


:Overlay
   .Curve.I   :Curve   [time]   (Snow Cube)
   .Scatter.I :Scatter   [time]   (s)

In [51]:
# Just filter to 1 year for publication
doy = int(tile_stats.loc[tile, 'smin_avg'])

# Filter and (if needed) Reindex
yr2 = int(snowCube_c.time.dt.year[-1]) # Last year represented in cube

# Convert day-of-year to filterable strings
if ((yr2 - 1) / 4).is_integer() == False: # Not a leap year
    date1 = dt.datetime(yr2 - 1, 1, 1) + dt.timedelta(doy) # Day after first year
if ((yr2 - 1) / 4).is_integer() == True: # Leap year
    date1 = dt.datetime(yr2 - 1, 1, 1) + dt.timedelta(doy + 1) # Day after first year
if (yr2 / 4).is_integer() == False: # Not a leap year (doy )
    date2 = dt.datetime(yr2, 1, 1) + dt.timedelta(doy - 1) # Day-of-year next year
if (yr2 / 4).is_integer() == True: # Leap year
    date2 = dt.datetime(yr2, 1, 1) + dt.timedelta(doy) # Day-of-year next year

start = f'{date1.year}-{date1.month:02d}-{date1.day:02d}'
end =  f'{date2.year}-{date2.month:02d}-{date2.day:02d}'

# Filter
snowCube_c = snowCube_c.sel(time = slice(start, end))
snowCube_c

<xarray.DataArray (time: 116, y: 10, x: 10)> Size: 46kB
array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]], dtype=float32)
Coordinates:
    band         <U5 20B 'Fmask'
  * x            (x) float64 80B 7.042e+05 7.042e+05 ... 7.044e+05 7.044e+05
  * y            (y) float64 80B 6.192e+05 6.192e+05 ... 6.189e+05 6.189e+05
  * time         (time) datetime64[ns] 928B 2022-08-16 2022-08-18 ... 2023-08-13
    spatial_ref  int64 8B 0

In [52]:
(snowCube_c[:, 5, 5].rename('Snow Cube') # To plot with hvplot
                    .dropna(dim = 'time') # Prepare for daily resample
                    .resample(time = '1D').interpolate('linear').astype('float32') # Linear daily resample
                    # Reindex to fill 2 year time-series?
                    .hvplot(height = 600, width = 1500) * # Drop na and interpolate na to see full picture
 
(snowCube_c[:, 5, 5].rename('s').hvplot.scatter(marker = 'o', color = 'black', size = 15))) # Observation points

:Overlay
   .Curve.I   :Curve   [time]   (Snow Cube)
   .Scatter.I :Scatter   [time]   (s)

In [53]:
#snowCube_c[:, 5, 5]

### dailySnowCube
- Find values for unclear observations (linear)
- Daily resample, linear
- Reindexing only if required

In [54]:
%%time

snowCube_c = snow_util.dailySnowCube(snowCube_c, doy = int(tile_stats.loc[tile, 'smin_avg']), verbose = True)
#snowCube_c

(snowCube_c[:, 5, 5].rename('Snow Cube') # To plot with hvplot
                    .hvplot(height = 600, width = 1200) * # Drop na and interpolate na to see full picture

(snowCube_c[:, 5, 5].rename('s').hvplot.scatter(marker = 'o', color = 'black', size = 15))) # Observation points

Filled in all NaNs in snowCube with linearly interpolated values in time.
Resampled to daily snowCube (2022-08-16 to 2023-08-13, n = 363) using linear interpolation.
Filtered to winter year (2022-08-16 to 2023-08-13, n = 363).
Reindexed to full year cube (2022-08-14 to 2023-08-13, n = 365).
Reinidexing not required since daily snowCube already contains all possible dates.
CPU times: total: 93.8 ms
Wall time: 540 ms


:Overlay
   .Curve.I   :Curve   [time]   (Snow Cube)
   .Scatter.I :Scatter   [time]   (s)

In [55]:
# Binary cube
snowCube_b = xr.where(snowCube_c >= 0.4999, 1, snowCube_c) # If no uncertainty (e.g., IMS - don't need this)
snowCube_b = xr.where(snowCube_c < 0.4999, 0, snowCube_b) # 0.4999 accounts for weird rounding in some cases

(snowCube_b[:, 5, 5].rename('Binary Snow Cube') # To plot with hvplot
                    .hvplot(height = 600, width = 2400) * # Drop na and interpolate na to see full picture

(snowCube_b[:, 5, 5].rename('s').hvplot.scatter(marker = 'o', color = 'black', size = 15))) # Observation points

:Overlay
   .Curve.I   :Curve   [time]   (Binary Snow Cube)
   .Scatter.I :Scatter   [time]   (s)

In [56]:
# Uncertainty cube
snowCube_u = xr.where((snowCube_c < 0.005) | (snowCube_c > 0.995), 0, snowCube_c)
snowCube_u = xr.where((snowCube_c >= 0.005) & (snowCube_c <= 0.995) , 1, snowCube_u) 

(snowCube_u[:, 5, 5].rename('Uncertainty Snow Cube') # To plot with hvplot
                    .hvplot(height = 600, width = 2400) * # Drop na and interpolate na to see full picture

(snowCube_u[:, 5, 5].rename('s').hvplot.scatter(marker = 'o', color = 'black', size = 15))) # Observation points

:Overlay
   .Curve.I   :Curve   [time]   (Uncertainty Snow Cube)
   .Scatter.I :Scatter   [time]   (s)

In [57]:
#snowCube_c[:,5,5]

### snowDynamics stats

In [58]:
%%time

snowDynamics = snow_util.dailySnowCube2SnowDynamics(snowCube_c, uncertainty = True, verbose = False)
#snowDynamics

print('Snow status: ' + str(snowDynamics['snow_status'][5,5].values))
print('Snow periods: ' + str(snowDynamics['snow_periods'][5,5].values))
print('Snow start: ' + str(snowDynamics['snow_start'][5,5].values))
print('Snow start uncertainty: ' + str(snowDynamics['snow_start_u'][5,5].values))
print('Snow end: ' + str(snowDynamics['snow_end'][5,5].values))
print('Snow end uncertainty: ' + str(snowDynamics['snow_end_u'][5,5].values))
print('Snow length: ' + str(snowDynamics['snow_length'][5,5].values))
print('Snow length uncertainty: ' + str(snowDynamics['snow_length_u'][5,5].values))

Snow status: 0
Snow periods: 1
Snow start: -53.0
Snow start uncertainty: 7.0
Snow end: 143.0
Snow end uncertainty: 4.0
Snow length: 197
Snow length uncertainty: 11.0
CPU times: total: 359 ms
Wall time: 2.42 s


### All at once
- For re-testing (e.g., after changing settings)

In [59]:
# %%time

# if np.isnan(tile_stats.loc[tile, 'send_last']) == True:
#     implausible_snow = []
# else: # Week buffer from latest recorded end date and earliest recorded start date
#     implausible_snow = [int(tile_stats.loc[tile, 'send_last']) + 7, int(tile_stats.loc[tile, 'sstart_first']) - 7]

# thresh = 2 # Originally 3 but in some situations leads to extra uncertainty
# x = 1686
# y = 1055

# snowCube_c = snow_util.cleanSnowCube(snowCube[:, x-5:x+5, y-5:y+5], implausible_snow = implausible_snow, thresh = thresh)
# snowCube_c = snow_util.dailySnowCube(snowCube_c, doy = int(tile_stats.loc[tile, 'smin_avg']), verbose = False)
# snowDynamics = snow_util.dailySnowCube2SnowDynamics(snowCube_c, uncertainty = True, verbose = False)

# print('Snow status: ' + str(snowDynamics['snow_status'][5,5].values))
# print('Snow periods: ' + str(snowDynamics['snow_periods'][5,5].values))
# print('Snow start: ' + str(snowDynamics['snow_start'][5,5].values))
# print('Snow start uncertainty: ' + str(snowDynamics['snow_start_u'][5,5].values))
# print('Snow end: ' + str(snowDynamics['snow_end'][5,5].values))
# print('Snow end uncertainty: ' + str(snowDynamics['snow_end_u'][5,5].values))
# print('Snow length: ' + str(snowDynamics['snow_length'][5,5].values))
# print('Snow length uncertainty: ' + str(snowDynamics['snow_length_u'][5,5].values))

# Testing

In [60]:
# path = r"C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\HBL\TemporalSnow\Fmask_V1\SingleTileExample\1819\clearObsPerDay_winterYear1819.nc"

# test = pputil.uploadNC(path, type = 'dataset')
# test

In [61]:
# doy = round(tile_stats.loc[tile, 'smin_avg'])
# doy

In [62]:
# try:
#     smin_doy = round(tile_stats.loc[tile, 'smin_avg'])
# except ValueError: # Nan values
#     smin_doy = round(tile_stats.loc[tile + 1, 'smin_avg']) # Just pick next tile 

# smin_doy

In [63]:
# try:
#     smax_doy = round(tile_stats.loc[tile, 'smax_avg'])
# except ValueError: # Nan values
#     smax_doy = round(tile_stats.loc[tile + 1, 'smax_avg']) # Just pick next tile 

# smax_doy

In [64]:
# try:
#     sstart_doy = round(tile_stats.loc[tile, 'sstart_avg'])
# except ValueError: # Nan values
#     if tile_stats.loc[tile, 'slen_avg'] > 0: # Perennial
#         sstart_doy = round(tile_stats.loc[tile, 'smin_avg'])
#     if tile_stats.loc[tile, 'slen_avg'] == 0: # Snow-free
#         print('Add NaN layer to dataset?')
#     else: # 
#         sstart_doy = round(tile_stats.loc[tile + 1, 'sstart_avg']) # Just pick next tile 

# sstart_doy

In [65]:
# try:
#     send_doy = round(tile_stats.loc[tile, 'send_avg'])
# except ValueError: # Nan values
#     if tile_stats.loc[tile, 'slen_avg'] > 0: # Perennial
#         send_doy = round(tile_stats.loc[tile, 'smin_avg'])
#     if tile_stats.loc[tile, 'slen_avg'] == 0: # Snow-free
#         print('Add NaN layer to dataset?')
#     else: # 
#         send_doy = round(tile_stats.loc[tile + 1, 'send_avg']) # Just pick next tile 

# send_doy

In [66]:
# wys_date = dt.datetime(int(snowCube.time.dt.year[0]), 1, 1) + dt.timedelta(smin_doy) # Day after first year
# wye_date = dt.datetime(int(snowCube.time.dt.year[-1]), 1, 1) + dt.timedelta(smin_doy) - dt.timedelta(1) # Day-of-year next year

# wys_str = f'{wys_date.year}-{wys_date.month:02d}-{wys_date.day:02d}'
# print(wys_str)
# wye_str =  f'{wye_date.year}-{wye_date.month:02d}-{wye_date.day:02d}'
# print(wye_str)

In [67]:
# opd_wy = snowCube.sel(time = slice(wys_str, wye_str)) # Filter to winter year
# # Count clear, divide by number of possible days
# opd_wy = (opd_wy.notnull().sum(dim = 'time') / ((wye_date - wys_date).days + 1)).astype('float32') 
# opd_wy

In [68]:
# sss_date = dt.datetime(int(snowCube.time.dt.year[0]), 12, 31) + dt.timedelta(sss_doy) # Snow start
# sse_date = dt.datetime(int(snowCube.time.dt.year[0]), 12, 31) + dt.timedelta(sse_doy) # Snow end

# sss_str = f'{sss_date.year}-{sss_date.month:02d}-{sss_date.day:02d}'
# print(sss_str)
# sse_str =  f'{sse_date.year}-{sse_date.month:02d}-{sse_date.day:02d}'
# print(sse_str)

In [69]:
# opd_ss = snowCube.sel(time = slice(sss_str, sse_str)) # Filter to snow season
# # Count clear, divide by number of possible days
# opd_ss = (opd_ss.notnull().sum(dim = 'time') / ((sse_date - sss_date).days + 1)).astype('float32') 
# opd_ss

In [70]:
# # End of snow fall (7 weeks later)
# sfe_doy = sss_doy + 48
# sfe_doy

In [71]:
# sfe_date = dt.datetime(int(snowCube.time.dt.year[0]), 12, 31) + dt.timedelta(sfe_doy) # Snow end

# sfe_str =  f'{sfe_date.year}-{sfe_date.month:02d}-{sfe_date.day:02d}'
# print(sss_str)
# print(sfe_str)

In [72]:
# opd_sf = snowCube.sel(time = slice(sss_str, sfe_str)) # Filter to snow season
# # Count clear, divide by number of possible days
# opd_sf = (opd_sf.notnull().sum(dim = 'time') / ((sfe_date - sss_date).days + 1)).astype('float32') 
# opd_sf

In [73]:
# sps_date = dt.datetime(int(snowCube.time.dt.year[0]), 12, 31) + dt.timedelta(smax_doy) - dt.timedelta(24) # 3.5 weeks before
# spe_date = dt.datetime(int(snowCube.time.dt.year[0]), 12, 31) + dt.timedelta(smax_doy) + dt.timedelta(24) # 3.5 weeks after

# sps_str = f'{sps_date.year}-{sps_date.month:02d}-{sps_date.day:02d}'
# print(sps_str)
# spe_str =  f'{spe_date.year}-{spe_date.month:02d}-{spe_date.day:02d}'
# print(spe_str)

In [74]:
# opd_sp = snowCube.sel(time = slice(sps_str, spe_str)) # Filter to snow season
# # Count clear, divide by number of possible days
# opd_sp = (opd_sp.notnull().sum(dim = 'time') / ((spe_date - sps_date).days + 1)).astype('float32') 
# opd_sp

In [75]:
# # Beginning of snow melt (7 weeks before)
# sms_doy = sse_doy - 48
# sms_doy

In [76]:
# sms_date = dt.datetime(int(snowCube.time.dt.year[0]), 12, 31) + dt.timedelta(sms_doy) # Snow start

# sms_str =  f'{sms_date.year}-{sms_date.month:02d}-{sms_date.day:02d}'
# print(sms_str)
# print(sse_str)

In [77]:
# opd_sm = snowCube.sel(time = slice(sms_str, sse_str)) # Filter to snow season
# # Count clear, divide by number of possible days
# opd_sm = (opd_sm.notnull().sum(dim = 'time') / ((sse_date - sms_date).days + 1)).astype('float32') 
# opd_sm

In [78]:
# tile_stats = pd.read_csv('C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/Canada/Snow Dynamics/IMS (1 km)/Outputs/tiles_IMS_ZonalSnowDynamics.csv', index_col = 0)
# tile_stats = tile_stats[['smin_avg', 'smax_avg', 
#                          'sstart_1819', 'sstart_1920', 'sstart_2021', 'sstart_2122', 'sstart_2223','sstart_avg', 
#                          'send_1819', 'send_1920', 'send_2021', 'send_2122', 'send_2223', 'send_avg', 
#                          #'slen_avg',  # Don't need actually
#                          'sPer_sum', 'sFree_sum',]] # Filter to columns of interest
# tile_stats

In [79]:
# # Fill in tiles without any IMS coverage with nearby values
# tile_stats.loc[400] = tile_stats.loc[401] # For 400, replace with 401
# tile_stats.loc[2002] = tile_stats.loc[2066] # For 2002, replace with 2066
# tile_stats.loc[2139] = tile_stats.loc[2138] # For 2139, replace with 2138
# tile_stats.loc[2932] = tile_stats.loc[2933] # For 2932, replace with 2933

# tile_stats[tile_stats['smin_avg'].isna()]

In [80]:
# # For perennial snow tiles, set as full year (no longer dropping snow free tiles)
# # drop = [] # Snow free tile indices to drop

# for tile in tile_stats.index:

#     # Fill in snow_start and snow_end for perennial snow tiles 
#     if (np.isnan(tile_stats.loc[tile, 'sstart_avg'])) & ((tile_stats.loc[tile, 'sPer_sum'] > 0)):
#         tile_stats.loc[tile, 'sstart_avg'] = tile_stats.loc[tile, 'smin_avg'] - 364
#         tile_stats.loc[tile, 'send_avg'] = tile_stats.loc[tile, 'smin_avg']   

# #     # Remove snow free tiles
# #     if (np.isnan(tile_stats.loc[tile, 'sstart_avg'])) & ((tile_stats.loc[tile, 'sFree_sum'] > 0)):
# #         drop.append(tile)

#     # Get latest snow end date (julian day) for all tiles
#     send_last = tile_stats.loc[tile, ['send_1819', 'send_1920', 'send_2021', 'send_2122', 'send_2223']].max()

#     if send_last < 100: # Must be at least day 100 (April 10)
#         send_last = 100
    
#     tile_stats.loc[tile, 'send_last'] = send_last

#     # Get earliest snow start date (julian day) fr all tiles
#     sstart_first = tile_stats.loc[tile, ['sstart_1819', 'sstart_1920', 'sstart_2021', 'sstart_2122', 'sstart_2223']].min() + 365

#     if sstart_first > 330: # Can't be later than day 330 (Nov 26)
#         sstart_first = 330
    
#     tile_stats.loc[tile, 'sstart_first'] = sstart_first

#     # For tiles with no recorded snow, provide send_last and sstart first
#     if (np.isnan(tile_stats.loc[tile, 'send_last'])) & ((tile_stats.loc[tile, 'sFree_sum'] > 0)):
#         tile_stats.loc[tile, 'send_last'] = 100
#         tile_stats.loc[tile, 'sstart_first'] = 330

#     # For tiles where sstart_first and ssend_last almost overlap... (Perennial)
#     if tile_stats.loc[tile, 'sstart_first'] - tile_stats.loc[tile, 'send_last'] <= 30:
#         tile_stats.loc[tile, 'send_last'] = np.nan
#         tile_stats.loc[tile, 'sstart_first'] = np.nan        
        
# # tile_stats = tile_stats.drop(drop)
# tile_stats = tile_stats.drop(columns = ['sPer_sum', 'sFree_sum', 
#                                         'send_1819', 'send_1920', 'send_2021', 'send_2122', 'send_2223',
#                                         'sstart_1819', 'sstart_1920', 'sstart_2021', 'sstart_2122', 'sstart_2223']) # Drop unneded columns
# tile_stats

In [81]:
# tile_stats['smin_avg'].hist()
# print(tile_stats['smin_avg'].min())
# print(tile_stats['smin_avg'].max())
# print(len(tile_stats[tile_stats['smin_avg'].isna()]))

In [82]:
# tile_stats['smax_avg'].hist()
# print(tile_stats['smax_avg'].min())
# print(tile_stats['smax_avg'].max())
# print(len(tile_stats[tile_stats['smax_avg'].isna()]))

In [83]:
# tile_stats['sstart_avg'].hist()
# print(tile_stats['sstart_avg'].min())
# print(tile_stats['sstart_avg'].max())
# print(len(tile_stats[tile_stats['sstart_avg'].isna()])) # Snow free tiles

In [84]:
# tile_stats['send_avg'].hist()
# print(tile_stats['send_avg'].min())
# print(tile_stats['send_avg'].max())
# print(len(tile_stats[tile_stats['send_avg'].isna()])) # Snow free tiles

In [85]:
# (tile_stats['send_avg'] - tile_stats['sstart_avg']).hist()
# print((tile_stats['send_avg'] - tile_stats['sstart_avg']).min())
# print((tile_stats['send_avg'] - tile_stats['sstart_avg']).max())
# print((tile_stats['send_avg'] - tile_stats['sstart_avg']).isna().sum()) # Snow free tiles

In [86]:
# tile_stats['send_last'].hist()
# print(tile_stats['send_last'].min())
# print(tile_stats['send_last'].max())
# print(len(tile_stats[tile_stats['send_last'].isna()])) # Perennial snow tiles

In [87]:
#tile_stats[(tile_stats['send_last'].isna()) & (~tile_stats['sstart_first'].isna())]

In [88]:
#tile_stats[tile_stats['send_last'] == tile_stats['send_last'].max()]

In [89]:
# tile_stats['sstart_first'].hist()
# print(tile_stats['sstart_first'].min())
# print(tile_stats['sstart_first'].max())
# print(len(tile_stats[tile_stats['sstart_first'].isna()])) # Perennial snow tiles

In [90]:
# (tile_stats['sstart_first'] - tile_stats['send_last']).hist()
# print((tile_stats['sstart_first'] - tile_stats['send_last']).min())
# print((tile_stats['sstart_first'] - tile_stats['send_last']).max())
# print((tile_stats['sstart_first'] - tile_stats['send_last']).isna().sum()) # Perennial snow tiles

In [91]:
# tile_stats[(tile_stats['sstart_first'] - tile_stats['send_last']) <= 40]

In [92]:
# tile_stats = tile_stats.round().astype('int16') # Round to nearest integer
# tile_stats

In [93]:
#tile_stats.to_csv('tiles_IMS_SnowDynamics4ObservationsPerDay.csv')

In [94]:
# %%time
# clearObsPerDay['winterYear'].rio.to_raster('clearObsPerDay_winterYear' + str(w_s)[2:] + str(w_e)[2:] + '.tif')
# clearObsPerDay['snowSeason'].rio.to_raster('clearObsPerDay_snowSeason' + str(w_s)[2:] + str(w_e)[2:] + '.tif')
# clearObsPerDay['snowFall'].rio.to_raster('clearObsPerDay_snowFall' + str(w_s)[2:] + str(w_e)[2:] + '.tif')
# clearObsPerDay['snowPeak'].rio.to_raster('clearObsPerDay_snowPeak' + str(w_s)[2:] + str(w_e)[2:] + '.tif')
# clearObsPerDay['snowMelt'].rio.to_raster('clearObsPerDay_snowMelt' + str(w_s)[2:] + str(w_e)[2:] + '.tif')